In [2]:
import ipdb
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pylab as pylab
import cartopy
from utils import constants as cnst
from eod import msg
from utils import u_grid, u_interpolate as u_int, constants as cnst, u_arrays, u_darrays, u_met
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
import cartopy.io.shapereader as shapereader
import pickle as pkl
import glob

coast = shapereader.natural_earth(resolution='110m',
                                  category='physical',
                                  name='coastline')

countries = shapereader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [14]:
filetag = 'pl'
era_hour = 12

In [4]:
#node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node3_MCS_matchTable.p', "rb"))
node_days = pd.read_csv(cnst.network_data + 'data/SOMS/saves/3x3_mergedNodes/node3_MCS_matchTable_mergedNodes_5mmMCS.csv', index_col=0, parse_dates=True)

In [5]:
pd.to_datetime(node_days.index)[0].replace(hour=5)

Timestamp('2004-04-09 05:00:00')

In [6]:
node_days

,prcp,q925,q650,t925,shear,month,hour,year,day,area,tmin,rh,tcwv,lat,lon
2004-04-09,7.707650,14.260949,5.994442,27.298914,8.651213,4.0,21.0,2004.0,9.0,11375.0,-69.812521,58.225094,44.268383,8.461831,-5.751067
2004-04-09,6.934962,14.963483,6.016386,24.061700,10.243200,4.0,21.0,2004.0,9.0,62025.0,-70.000000,73.759201,42.655621,8.505196,-8.391715
2004-04-18,30.334374,15.645405,4.486968,24.681482,15.065556,4.0,18.0,2004.0,18.0,38600.0,-91.604637,74.144287,45.452923,7.434747,1.951620
2004-04-18,6.826766,14.024221,3.300007,28.126855,10.045700,4.0,18.0,2004.0,18.0,8825.0,-80.500711,54.635201,35.699654,11.119935,-0.157512
2004-04-18,11.059868,16.948735,5.510681,21.492456,15.331886,4.0,18.0,2004.0,18.0,15025.0,-71.814140,97.718216,55.644005,5.535386,5.598618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-06-18,23.893955,15.521908,3.021870,26.827997,17.114223,6.0,20.0,2015.0,18.0,33250.0,-88.610683,64.988579,39.765556,10.127258,-6.995880
2015-06-18,6.483023,16.053395,3.671936,22.937341,13.490683,6.0,20.0,2015.0,18.0,10225.0,-79.968680,84.734657,42.029091,11.023351,-12.796294
2015-06-18,23.000604,15.491045,4.147871,29.189233,22.687401,6.0,21.0,2015.0,18.0,118275.0,-85.065875,56.279160,39.375114,11.240192,-5.836265
2015-06-18,19.034942,14.920573,3.688829,23.057520,19.081163,6.0,21.0,2015.0,18.0,34025.0,-73.305625,78.165268,46.607536,7.136541,-2.880156


In [15]:
def save_node_clim(filetag, node, era_hour):
    
    nodestr = 'node'+str(node)
    node_days = pd.read_csv(cnst.network_data + 'data/SOMS/saves/3x3_mergedNodes/'+nodestr+'_MCS_matchTable_mergedNodes_5mmMCS.csv', index_col=0, parse_dates=True)
    #node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/3x3/MCStable_3x3_5-9N/'+nodestr+'_MCS_matchTable.p', "rb"))
    
    #### potentially reduce MCS box here
    region = (node_days.lat>5) & (node_days.lat<8) & (node_days.lon>-10) & (node_days.lon<10)
    node_days = node_days[region]
    
    node_days_era = []
    for nd in node_days.index:
        node_days_era.append(nd.replace(hour=era_hour))
    node_days_era = pd.Series(node_days_era).values
        
    if filetag == 'pl':
        fft = 'pressure_levels'
    else:
        fft = 'surface'
        
    filebase = cnst.elements_drive + '/Africa/WestAfrica/ERA5/hourly/' 
    filepath = filebase + fft+'/ERA5*'+filetag+'.nc'
    out = '/home/ck/DIR/cornkle/data/SOMS/saves/3x3_mergedNodes/'+nodestr+'_ERA5_5MCS_5mm_mean_5-8N_'+filetag+'.nc'
    outdev = '/home/ck/DIR/cornkle/data/SOMS/saves/3x3_mergedNodes/'+nodestr+'_ERA5_5MCS_5mm_stddev_5-8N_'+filetag+'.nc'
    
    def means(da, node_days):
        da = da.sel(time=(da['time.hour']==era_hour))
        try:
            da = da.sel(level=[925,850,750,650,550,250])
        except:
            pass
        
        if (filetag == 'pl'):
            if not 'z' in da.data_vars:
                ipdb.set_trace()
                
        dal = []
        
        u, inv = np.unique(node_days, return_inverse=True)
        n = np.bincount(inv)
        goodinds = u[(n>=5)] #minimum 5 MCS per MCS day
        
        intersect_dates = np.intersect1d(goodinds, da.time.values)
        if len(intersect_dates) == 0:
            return None
        try:
            da = da.sel(time=intersect_dates)
        except:
            return None

        da = u_darrays.flip_lat(da)
        return da
    
    srfc_list = []
    
    for f in glob.glob(filepath):
        print('Doing', f)
        srfc = xr.open_dataset(f) 
        srfc_concat = means(srfc,node_days_era)
        if srfc_concat is None:
            continue

        srfc_list.append(srfc_concat)
    
    full = xr.concat(srfc_list, dim='time')
    #ipdb.set_trace()
    mean = full.mean('time')
    dev = full.std('time')
    
    mean.to_netcdf(out)
    dev.to_netcdf(outdev)

In [16]:
for ni in range(1,10):
#     nodestr='node'+str(1)
#     all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
#     node_days = all_node_file[nodestr]
#     node_days_era = []
#     for nd in node_days:
#         node_days_era.append(nd.replace(hour=era_hour))
        
    save_node_clim(filetag, ni, era_hour)

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1980_11_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2012_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2012_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1981_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1982_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1982_02_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2003_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2004_05_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1996_12_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1986_12_pl.nc

...

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2009_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2010_06_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2017_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2018_01_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_02_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_03_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2007_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_1991_12_pl.nc

Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_04_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_05_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_06_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_07_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_08_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_09_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_10_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_11_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2000_12_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2001_01_pl.nc
Doing /media/ck/Elements//Africa/WestAfrica/ERA5/hourly/pressure_levels/ERA5_2001_02_pl.nc

FileNotFoundError: [Errno 2] No such file or directory: '/home/ck/DIR/cornkle/data/SOMS/saves/3x3_mergedNodes/node7_MCS_matchTable_mergedNodes_5mmMCS.csv'

In [ ]:
'z' in da